In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/03 02:56:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# recursively read all data under green/2021/01
df_green = spark.read \
    .option("header", "true") \
    .csv('data/raw/green/2021/01/')

In [4]:
df_green.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2021-01-01 00:15:56|  2021-01-01 00:19:52|                 N|         1|          43|         151|              1|         1.01|        5.5|  0.5|    0.

In [5]:
df_green.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- lpep_pickup_datetime: string (nullable = true)
 |-- lpep_dropoff_datetime: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- extra: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- ehail_fee: string (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- trip_type: string (nullable = true)
 |-- congestion_surcharge: string (nullable = true)



In [25]:
import pandas as pd

In [26]:
df_green_pd = pd.read_csv("data/raw/green/2021/01/green_tripdata_2021-01.csv.gz", nrows=1000)

In [27]:
df_green_pd.dtypes

VendorID                   int64
lpep_pickup_datetime      object
lpep_dropoff_datetime     object
store_and_fwd_flag        object
RatecodeID                 int64
PULocationID               int64
DOLocationID               int64
passenger_count            int64
trip_distance            float64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
ehail_fee                float64
improvement_surcharge    float64
total_amount             float64
payment_type               int64
trip_type                  int64
congestion_surcharge     float64
dtype: object

In [28]:
spark.createDataFrame(df_green_pd).schema

StructType([StructField('VendorID', LongType(), True), StructField('lpep_pickup_datetime', StringType(), True), StructField('lpep_dropoff_datetime', StringType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('RatecodeID', LongType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('passenger_count', LongType(), True), StructField('trip_distance', DoubleType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('ehail_fee', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('payment_type', LongType(), True), StructField('trip_type', LongType(), True), StructField('congestion_surcharge', DoubleType(), True)])

In [29]:
from pyspark.sql.types import StructType, StructField, LongType, StringType, DoubleType, IntegerType, TimestampType

In [30]:
green_schema = StructType([StructField('VendorID', IntegerType(), True), 
            StructField('lpep_pickup_datetime', TimestampType(), True), 
            StructField('lpep_dropoff_datetime', TimestampType(), True), 
            StructField('store_and_fwd_flag', StringType(), True), 
            StructField('RatecodeID', IntegerType(), True), 
            StructField('PULocationID', IntegerType(), True), 
            StructField('DOLocationID', IntegerType(), True), 
            StructField('passenger_count', IntegerType(), True), 
            StructField('trip_distance', DoubleType(), True), 
            StructField('fare_amount', DoubleType(), True), 
            StructField('extra', DoubleType(), True), 
            StructField('mta_tax', DoubleType(), True), 
            StructField('tip_amount', DoubleType(), True), 
            StructField('tolls_amount', DoubleType(), True), 
            StructField('ehail_fee', DoubleType(), True), 
            StructField('improvement_surcharge', DoubleType(), True), 
            StructField('total_amount', DoubleType(), True), 
            StructField('payment_type', IntegerType(), True), 
            StructField('trip_type', IntegerType(), True), 
            StructField('congestion_surcharge', DoubleType(), True)])

In [31]:
df_green = spark.read \
    .option("header", "true") \
    .schema(green_schema) \
    .csv('data/raw/green/2021/01/')

In [ ]:
df_green \
    .repartition(4) \
    .write.parquet('data/pq/green/2021/01/')

In [32]:
df_green.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- trip_type: integer (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [33]:
df_yellow_pd = pd.read_csv("data/raw/yellow/2021/01/yellow_tripdata_2021-01.csv.gz", nrows=1000)

In [34]:
spark.createDataFrame(df_yellow_pd).schema

StructType([StructField('VendorID', LongType(), True), StructField('tpep_pickup_datetime', StringType(), True), StructField('tpep_dropoff_datetime', StringType(), True), StructField('passenger_count', LongType(), True), StructField('trip_distance', DoubleType(), True), StructField('RatecodeID', LongType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('payment_type', LongType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True)])

In [35]:
yellow_schema = StructType([StructField('VendorID', IntegerType(), True), 
            StructField('tpep_pickup_datetime', TimestampType(), True), 
            StructField('tpep_dropoff_datetime', TimestampType(), True), 
            StructField('passenger_count', IntegerType(), True), 
            StructField('trip_distance', DoubleType(), True), 
            StructField('RatecodeID', IntegerType(), True), 
            StructField('store_and_fwd_flag', StringType(), True), 
            StructField('PULocationID', IntegerType(), True), 
            StructField('DOLocationID', IntegerType(), True), 
            StructField('payment_type', LongType(), True), 
            StructField('fare_amount', DoubleType(), True), 
            StructField('extra', DoubleType(), True), 
            StructField('mta_tax', DoubleType(), True), 
            StructField('tip_amount', DoubleType(), True), 
            StructField('tolls_amount', DoubleType(), True), 
            StructField('improvement_surcharge', DoubleType(), True), 
            StructField('total_amount', DoubleType(), True), 
            StructField('congestion_surcharge', DoubleType(), True)])

In [36]:
df_yellow = spark.read \
    .option("header", "true") \
    .schema(yellow_schema) \
    .csv('data/raw/yellow/2021/01/')

In [42]:
from pyspark.sql import SparkSession
import os
from glob import glob

# Initialize Spark session
# spark = SparkSession.builder \
#     .appName("NYC Taxi Data Processing") \
#     .getOrCreate()

def process_all_data_at_once(taxi_type, yrs=None):
    """
    Approach 1: Read all data at once, partition by year/month, and save
    
    Args:
        taxi_type (str): 'green' or 'yellow'
        yrs (List[str]): List of years to process (e.g., ['2020', '2021']) or None for all years
    """
    # Create path pattern based on whether specific years are provided
    if yrs is not None and len(yrs) > 0:
        # Create a list of paths for each specified year
        paths = [f"data/raw/{taxi_type}/{year}/*/*/{taxi_type}_tripdata_{year}-{month}.csv.gz" for year in yrs]
        print(f"Reading {taxi_type} taxi data for years: {', '.join(yrs)}...")
    else:
        # Use a single path pattern to read all years
        paths = [f"data/raw/{taxi_type}/*/*/*/{taxi_type}_tripdata_{year}-{month}.csv.gz"]
        print(f"Reading all {taxi_type} taxi data...")
    
    # Read all CSV files for the specified taxi type and years
    df = spark.read \
        .option("header", "true") \
        .schema(get_schema(taxi_type)) \
        .csv(paths)
    
    # Add year and month columns based on input_file_name
    df = df.withColumn("input_file", spark.sql.functions.input_file_name())
    df = df.withColumn("year", spark.sql.functions.regexp_extract("input_file", r"data/raw/\w+/(\d{4})/\d{2}/", 1))
    df = df.withColumn("month", spark.sql.functions.regexp_extract("input_file", r"data/raw/\w+/\d{4}/(\d{2})/", 1))
    
    # Write to Parquet partitioned by year and month
    print(f"Writing {taxi_type} taxi data to parquet with year/month partitioning...")
    df.write \
        .partitionBy("year", "month") \
        .mode("overwrite") \
        .parquet(f"data/pq/{taxi_type}")
    
    print(f"Completed processing {taxi_type} taxi data")

def process_month_by_month(taxi_type, yrs=None, num_partitions=4):
    """
    Approach 2: Process data month by month and repartition each month
    
    Args:
        taxi_type (str): 'green' or 'yellow'
        yrs (List[str]): List of years to process (e.g., ['2020', '2021']) or None for all years
        num_partitions (int): Number of partitions for each month's data
    """
    # Find all years and months
    base_path = f"data/raw/{taxi_type}"
    years_months = []
    
    # Find all year directories
    year_dirs = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]

    # Filter years if specified
    if yrs is not None and len(yrs) > 0:
        year_dirs = [d for d in year_dirs if d in yrs]
        print(f"Processing {taxi_type} taxi data for years: {', '.join(year_dirs)}")
    else:
        print(f"Processing all {taxi_type} taxi data years")
    
    for year in year_dirs:
        # Find all month directories in this year
        month_dirs = [d for d in os.listdir(os.path.join(base_path, year)) 
                     if os.path.isdir(os.path.join(base_path, year, d))]
        
        for month in month_dirs:
            years_months.append((year, month))
    
    # Process each year/month separately
    for year, month in years_months:
        print(f"Processing {taxi_type} taxi data for {year}/{month}...")
        
        # Read data for this specific year/month
        path = f"data/raw/{taxi_type}/{year}/{month}/{taxi_type}_tripdata_{year}-{month}.csv.gz"
        
        df = spark.read \
            .option("header", "true") \
            .schema(get_schema(taxi_type)) \
            .csv(path)
        
        # Repartition the dataframe
        df = df.repartition(num_partitions)
        
        # Write to Parquet
        output_path = f"data/pq/{taxi_type}/{year}/{month}"
        
        print(f"Writing {taxi_type} taxi data to {output_path} with {num_partitions} partitions...")
        df.write \
            .mode("overwrite") \
            .parquet(output_path)
        
        print(f"Completed processing {taxi_type} taxi data for {year}/{month}")

def get_schema(taxi_type):
    """
    Return the schema for the specified taxi type
    
    Args:
        taxi_type (str): 'green' or 'yellow'
    
    Returns:
        pyspark.sql.types.StructType: Schema for the specified taxi type
    """
    if taxi_type == "green":
        # Return green taxi schema that we've already created
        return green_schema
    elif taxi_type == "yellow":
        # Return yellow taxi schema that we've already created
        return yellow_schema
    else:
        raise ValueError(f"Unknown taxi type: {taxi_type}")


if __name__ == "__main__":
    # Specify the taxi type to process
    taxi_type = "green"
    
    # Specify years to process (or set to None for all years)
    years_to_process = ["2020", "2021"]  # Or set to None to process all years
    
    # Choose which approach to use
    approach = "month_by_month"  # or "all_at_once"
    
    if approach == "all_at_once":
        process_all_data_at_once(taxi_type, yrs=years_to_process)
    else:
        process_month_by_month(taxi_type, yrs=years_to_process, num_partitions=4)
    
    # Process the other taxi type if needed
    # taxi_type = "yellow"
    # process_all_data_at_once(taxi_type, yrs=years_to_process)  # or process_month_by_month(taxi_type, yrs=years_to_process)
    
    # Stop Spark session
    #spark.stop()

Processing yellow taxi data for years: 2020, 2021
Processing yellow taxi data for 2020/11...
Writing yellow taxi data to data/pq/yellow/2020/11 with 4 partitions...


Completed processing yellow taxi data for 2020/11
Processing yellow taxi data for 2020/08...
Writing yellow taxi data to data/pq/yellow/2020/08 with 4 partitions...


Completed processing yellow taxi data for 2020/08
Processing yellow taxi data for 2020/04...
Writing yellow taxi data to data/pq/yellow/2020/04 with 4 partitions...


Completed processing yellow taxi data for 2020/04
Processing yellow taxi data for 2020/01...
Writing yellow taxi data to data/pq/yellow/2020/01 with 4 partitions...


Completed processing yellow taxi data for 2020/01
Processing yellow taxi data for 2020/07...
Writing yellow taxi data to data/pq/yellow/2020/07 with 4 partitions...


...

Completed processing yellow taxi data for 2021/07
Processing yellow taxi data for 2021/05...
Writing yellow taxi data to data/pq/yellow/2021/05 with 4 partitions...


Completed processing yellow taxi data for 2021/05
Processing yellow taxi data for 2021/06...
Writing yellow taxi data to data/pq/yellow/2021/06 with 4 partitions...


Completed processing yellow taxi data for 2021/06
Processing yellow taxi data for 2021/02...
Writing yellow taxi data to data/pq/yellow/2021/02 with 4 partitions...


Completed processing yellow taxi data for 2021/02
Processing yellow taxi data for 2021/03...
Writing yellow taxi data to data/pq/yellow/2021/03 with 4 partitions...


Completed processing yellow taxi data for 2021/03
